In [1]:
import numpy as np
import pandas as pd
print(np.__version__)

1.21.6


In [2]:
# Create a basic dataset
df = pd.DataFrame([[8, 8, 4], [7, 9, 5], [6, 10, 6], [5, 12, 7]], columns=['cgpa', 'resumeScore', 'package'])
X = df.iloc[:, 0:2].values
y = df.iloc[:, -1].values
print(X)
print(y)

[[ 8  8]
 [ 7  9]
 [ 6 10]
 [ 5 12]]
[4 5 6 7]


In [3]:
def initialize_weights(nnDims):
    locWeights = {}
    L = len(nnDims)
    for l in range(1, L):
        locWeights['W' + str(l)] = np.ones((nnDims[l-1], nnDims[l])) * 0.1
        locWeights['B' + str(l)] = np.zeros(nnDims[l])

    return locWeights

In [4]:
def linear_forward(aPrev, W, B):
    return (np.dot(W.T, aPrev) + B)

def forward_propogate(Xi, weights):

    # Half the size of W,b dictionary
    L = len(weights) // 2 

    # a[0] = input = Xi
    A = Xi

    for l in range(1, L + 1):
        APrev = A
        Wl = weights['W' + str(l)]
        Bl = weights['B' + str(l)]
        # print('a[' + str(l-1) + ']: ', APrev)
        # print('W[' + str(l) + ']: ', Wl)
        # print('B[' + str(l) + ']: ', Bl)
        # print('--'*20)

        A = linear_forward(APrev, Wl, Bl)
        # print('a['+str(l)+']: ', A)
        # print('**'*20)

    return A, APrev

# weights = initialize_weights([2, 2, 1])
# yHat, A1 = forward_propogate(X[0], weights)
# print(yHat.shape)
# print(A1.shape)


In [5]:
def update_weights(weights, learningRate, y, yHat, A1, Xi):
    # using gradient descent
    weights['W2'][0][0] = weights['W2'][0][0] + learningRate * 2 * (y - yHat) * A1[0]
    weights['W2'][1][0] = weights['W2'][1][0] + learningRate * 2 * (y - yHat) * A1[1]
    weights['B2'][0] = weights['B2'][0] + learningRate * 2 * (y - yHat)

    weights['W1'][0][0] = weights['W1'][0][0] + learningRate * 2 * (y - yHat) * weights['W2'][0][0] * Xi[0]
    weights['W1'][1][0] = weights['W1'][1][0] + learningRate * 2 * (y - yHat) * weights['W2'][0][0] * Xi[1]
    weights['B1'][0] = weights['B1'][0] + learningRate * 2 * (y - yHat) * weights['W2'][0][0]

    weights['W1'][0][1] = weights['W1'][0][1] + learningRate * 2 * (y - yHat) * weights['W2'][1][0] * Xi[0]
    weights['W1'][1][1] = weights['W1'][1][1] + learningRate * 2 * (y - yHat) * weights['W2'][1][0] * Xi[1]
    weights['B1'][1] = weights['B1'][1] + learningRate * 2 * (y - yHat) * weights['W2'][1][0]

In [6]:
# Train the neural network
epochs = 5
learningRate = 0.001
nnDims = [2, 2, 1]
weights = initialize_weights(nnDims)

# for each epoch
for step in range(epochs):
    # create an empty array to save loss for each student
    loss = []
    # loop through each student and perform forward/back propogation
    for i in range(X.shape[0]):
        yHat, A1 = forward_propogate(X[i], weights)
        loss.append((y[i] - yHat[0]) ** 2)
        update_weights(weights, learningRate, y[i], yHat[0], A1, X[i])

    print('Epoch - ',i+1,'Loss - ',np.array(loss).mean())

weights       


Epoch -  4 Loss -  26.249691774703265
Epoch -  4 Loss -  19.19634375189233
Epoch -  4 Loss -  9.702531836210806
Epoch -  4 Loss -  3.0274162602080916
Epoch -  4 Loss -  1.0597082587765176


{'W1': array([[0.27790431, 0.27790431],
        [0.40579137, 0.40579137]]),
 'B1': array([0.0295198, 0.0295198]),
 'W2': array([[0.46411039],
        [0.46411039]]),
 'B2': array([0.11664796])}

## Let's try the same using Tensorflow and Keras

In [27]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

import matplotlib.pyplot as plt

In [19]:
model = Sequential()
model.add(Dense(2, activation='linear', input_dim=2))
model.add(Dense(1, activation='linear'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 2)                 6         
                                                                 
 dense_9 (Dense)             (None, 1)                 3         
                                                                 
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.set_weights([np.array([[0.1 , 0.1],
        [0.1,  0.1]], dtype=np.float32),
 np.array([0., 0.], dtype=np.float32),
 np.array([[0.1],
        [0.1]], dtype=np.float32),
 np.array([0.], dtype=np.float32)])

model.get_weights()

[array([[0.1, 0.1],
        [0.1, 0.1]], dtype=float32),
 array([0., 0.], dtype=float32),
 array([[0.1],
        [0.1]], dtype=float32),
 array([0.], dtype=float32)]

In [23]:
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='mean_squared_error', optimizer=optimizer)

In [26]:
model.fit(X, y, epochs=60, verbose=1, batch_size=1)

Epoch 1/60
4/4 [==============================] - 0s 3ms/step - loss: 0.1404
Epoch 2/60
4/4 [==============================] - 0s 3ms/step - loss: 0.1413
Epoch 3/60
4/4 [==============================] - 0s 4ms/step - loss: 0.1340
Epoch 4/60
4/4 [==============================] - 0s 3ms/step - loss: 0.1265
Epoch 5/60
4/4 [==============================] - 0s 3ms/step - loss: 0.1319
Epoch 6/60
4/4 [==============================] - 0s 4ms/step - loss: 0.1177
Epoch 7/60
4/4 [==============================] - 0s 4ms/step - loss: 0.1123
Epoch 8/60
4/4 [==============================] - 0s 4ms/step - loss: 0.1144
Epoch 9/60
4/4 [==============================] - 0s 4ms/step - loss: 0.1131
Epoch 10/60
4/4 [==============================] - 0s 4ms/step - loss: 0.1029
Epoch 11/60
4/4 [==============================] - 0s 5ms/step - loss: 0.0998
Epoch 12/60
4/4 [==============================] - 0s 4ms/step - loss: 0.0940
Epoch 13/60
4/4 [==============================] - 0s 4ms/step - loss: 0.